<h1>Exkurs: Koroutinen</h1>
<p>Wie angekündigt wird der Zugriff auf die Datenbank über ein <i>Repository</i> stattfinden. Dieses verwaltet den Zugriff auf die Datenbank und später das Internet. Da diese Operationen einige Zeit in Anspruch nehmen können, werden  beziehungsweise müssen diese asynchron durchgeführt werden. In Kotlin stehen dafür <i>Coroutines</i> bereit, die in diesem Exkurs rudimentär eingeführt werden. Die Anweisungen führt ein neuer Thread aus, der entweder eigenständig oder mit dem Hauptthread interagieren kann. Ein Vorteil ist, dass der Hauptthread durch die Operationen nicht blockiert wird und seine Berechnungen weiterführen kann. Um sie in Notebooks zu verwenden, muss zunächst die zugehörige Bibliothek importiert werden:</p>

<blockquote><code>@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.0")
import kotlinx.coroutines.*</code></blockquote>

<p>Es folgt ein einfaches Einführungsbeispiel:</p>

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.0")
import kotlinx.coroutines.*

GlobalScope.launch { //Startet die Koroutine
    var i = 0
    while(i < 5){
        println("Coroutine $i")
        delay(100L)
        i++
    }
}
println("Hauptthread")
Thread.sleep(2000L) //Hauptthread muss warten, da sonst die Koroutine vorzeitig abgebrochen wird

Hauptthread
Coroutine 0
Coroutine 1
Coroutine 2
Coroutine 3
Coroutine 4


<p>Es wird vom Hauptthread eine Koroutine mit <code>GlobalScope.launch</code> gestartet. In dieser wird in einer Schleife fünf Mal ein Text auf der Konsole ausgegeben. Der Hauptthread <i>schläft</i> zum Schluss kurz, sodass die Koroutine noch beendet wird.<br />
Es sind aber bereits schon einige Fachbegriffe gefallen, die hier näher erklärt werden:</p>
<ul>
    <li><b>Threads</b> werden von der Java Virtual Machine (JVM) verwaltet und verwenden die Multi-Threading-Funktionen des Betriebssystems. Sie besitzen zudem einen eigenen Stack-Speicher. Gemeinsame Informationen werden über einen Heap-Speicher mit anderen Threads geteilt.</li>
    <li><b>Koroutinen</b> werden von Kotlin verwaltet und sind nicht an einen konkreten CPU gebunden, sondern können beliebig verschoben werden. Beispielsweise wird eine Koroutine zuerst auf Kern 1 gestartet und dann auf dem Dritten beendet. Sie sind deutlich flexibler als Threads. Zudem besitzen sie keinen eigenen Speicher und sind ressourcensparsamer. Im Hintergund werden sie auf einer geringen Anzahl an Threads ausgeführt.</li>
    <li>Ein <b>Scope</b> ist der Kontext einer Koroutine. <i>GlobalScope</i> ist beispielsweise die <i>Top-Level</i>-Ebene.</li>
    <li>Der <b>Dispatcher</b> ist mit dem <i>Scope</i> verbunden und entscheidet von welchem Thread die Koroutine ausgeführt wird. Außerdem lässt sich aus diesem der Aufgabenbereich ableiten:</li>
</ul>

In [33]:
import kotlinx.coroutines.*

runBlocking {
    launch { // Besitzt den Kontext des Main-Treads
        println("main runBlocking      : Thread-Name:  ${Thread.currentThread().name}")
    }
    launch(Dispatchers.IO) { // Wird verwendet, wenn im Hintergund auf Dateien oder Netzwerk-Ressourcen zugegriffen wird
        println("IO                    : Thread-Name:  ${Thread.currentThread().name}")
    }
    launch(Dispatchers.Unconfined) { // Verwendet den aktuellen Thread
        println("Unconfined            : Thread-Name:  ${Thread.currentThread().name}")
    }
    launch(Dispatchers.Default) { // Für CPU-intensive Aufgaben
        println("Default               : Thread-Name:  ${Thread.currentThread().name}")
    }
    launch(newSingleThreadContext("eigenerThread")) { // Wird von einem eigenen Thread ausgeführt
        println("newSingleThreadContext: Thread-Name:  ${Thread.currentThread().name}")
    }
}

IO                    : Thread-Name:  DefaultDispatcher-worker-1
Unconfined            : Thread-Name:  Thread-138
Default               : Thread-Name:  DefaultDispatcher-worker-1
main runBlocking      : Thread-Name:  Thread-138
newSingleThreadContext: Thread-Name:  eigenerThread


StandaloneCoroutine{Completed}@1b7c473a

<p>Gestartet können Koroutinen mit dem Schlüsselwort <i>launch</i>, welches üblicherweise auf einem Kontext aufgerufen wird. Zusätzlich kann der <i>Dispatcher</i> bestimmt werden. Bei keiner Angabe, wird der <i>Default-Dispatcher</i> verwendet. Eine weitere Möglichkeit ist das Verwenden von <i>launch</i> in <i>runBlocking</i>, das den aktuellen Thread stoppt und nach Beendigung der Koroutinen wieder aktiviert.</p>

In [8]:
print("Anfang")
runBlocking {
    repeat(500) { //wird 500 mal ausgeführt
        launch { //startet eine Koroutine
            print(".")
        }
    }
}
print("Ende")

Anfang....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Ende

<p>Mit <i>join</i> kann auf eine gestarteten Koroutine gewartet werden.</p>

In [41]:
import kotlinx.coroutines.*

val job = GlobalScope.launch { //Startet die Koroutine und weist sie der Variable job zu
        delay((500..10000L).random())
        println("Fertig")
}

runBlocking { //Wartet auf die Koroutine
    job.join()
}

println("Koroutine beendet")

Fertig
Koroutine beendet


<p>Soll eine Koroutine ein Ergebnis zurückgeben, findet statt <i>launch async</i> Anwendung:</p>

In [9]:
import kotlinx.coroutines.*

val text = "Kotlin ist Toll"
val laenge = GlobalScope.async(Dispatchers.IO){
    delay(500L) //Wird diese Zeile einkommentiert, wirft getComplete() keinen Fehler
    text.length
}

println("Typ vom laenge: ${laenge::class.simpleName}")

try{
  println("laenge.getCompleted(): ${laenge.getCompleted()}")  
}
catch (e: Exception){
    println("Fehler: $e")
}

runBlocking {
    println("laenge: ${laenge.await()}")
}

Typ vom laenge: DeferredCoroutine
Fehler: java.lang.IllegalStateException: This job has not completed yet
laenge: 15


<p>Die Rückgabe ist vom Typ <i>Deferred&#60;Int></i>. Zusätzlich zum Wert ist gespeichert, ob die Koroutine beendet ist oder nicht. Mit <i>await()</i> in Verbindung mit <i>runBlocking</i> oder <i>getCompleted()</i> kann auf das Ergebnis gewartet werden. Zweiteres löst aber eine <i>IllegalStateException</i> aus, falls die Berechnung noch läuft.</p>

<p>Mit <i>cancel()</i> kann eine <i>Coroutine</i> abgebrochen werden:</p>

In [59]:
import kotlinx.coroutines.*

val job = GlobalScope.launch { //Startet die Koroutine
        delay((500..10000L).random())
        println("Fertig")
}

job.cancel()
println("Koroutine abgebrochen")

Koroutine beendet


<p>Bis jetzt wurde der Code der Koroutine immer in einem Lambda-Ausdruck angegeben. Dieser wird immer benötigt, jedoch können die Zeilen in eigene Funktionen geschoben werden. Diese müssen mit dem Schlüsselwort <i>suspend</i> gekennzeichnet sein.</p>

In [14]:
import kotlinx.coroutines.*

suspend fun fak(a: Int): Int{
    var result = 1
    var i = a

    while(i > 1){
        result *= i
        i--
    }
    return result
}

runBlocking{
    val result = async { //Startet die Koroutine
        fak(5)
    }
    println("fak(5): ${result.await()}")
}


fak(5): 120
